In [1]:
# ###############################################################
#       d888888b                         d888888b
#    d888    8888b                    d888888   888b
#  d88    88  898888b               d8888  888     88b
# d8P        88888888b             d88888888888     b8b
# 88        8888888888             88888888888       88
# 88       88888888888             8888888888        88
# 98b     88888888888P             988888888        d8P
#  988     888  8888P      _=_      9888898  88    88P
#    9888   888888P      q(-_-)p       98888    888P
#       9888888P         '_) (_`         9888888P
#          88            /__/  \            88
#          88          _(<_   / )_          88
#         d88b        (__\_\_|_/__)        d88b
###
###                  佛祖保佑，Bug退去；
###                  阿弥陀佛，善哉善哉
# ###############################################################

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [3]:
### 图像预处理
def preprocess(file_name):
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    chs = cv2.split(img)
    hue = ((chs[0] > 30) * 1) * chs[0]
    res = ((hue < 50) * 1) * hue

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    res = cv2.morphologyEx(res.astype(np.float32), cv2.MORPH_ERODE, kernel)
    resize = cv2.resize(res, (196, 196))
    resize = resize.reshape((1, 196, 196))
    resize = cv2.normalize(resize, resize, 0, 255, cv2.NORM_MINMAX)

    cv2.imwrite("test.jpg", resize[0])

    return resize

In [4]:
train_path = '/media/lor/data/data/plant-seedlings-classification/train'
train_samples = []
train_labels = []
label_to_idx = {}
for (i, cat) in enumerate(os.listdir(train_path)):
    label_to_idx[cat] = i
    cat_path = os.path.join(train_path, cat)
#     print(cat_path)
    for item in os.listdir(cat_path):
        train_labels.append(i)
        train_samples.append(os.path.join(cat_path, item))
print(len(train_labels))
print(len(train_samples))

4750
4750


In [24]:
class PlantDS(torch.utils.data.Dataset):
    def __init__(self, samples, targets):
        self.samples = samples
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        target = self.targets[idx]
        sample = preprocess(self.samples[idx])
        sample = sample / 255
        return (target, sample)

In [25]:
train_ds = PlantDS(train_samples, train_labels)
indices = list(range(train_ds.__len__()))
np.random.shuffle(indices)
train_indices = indices[0:4000]
valid_indices = indices[4000:4750]
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_indices)

In [26]:
train_dloader = torch.utils.data.DataLoader(train_ds, batch_size=16, num_workers=4, sampler=train_sampler)
valid_dloader = torch.utils.data.DataLoader(train_ds, batch_size=1, num_workers=4, sampler=valid_sampler)

In [27]:
class PlantNet(nn.Module):
    def __init__(self):
        super(PlantNet, self).__init__()
        self.layer_1 = nn.Sequential(
                        nn.Conv2d(1, 6, kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(6),
                        nn.ReLU(),
                        nn.Conv2d(6, 16, kernel_size=(5, 5), stride=1, padding=2),
                        nn.BatchNorm2d(16),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2),
        )
        self.layer_2 = nn.Sequential(
                        nn.Conv2d(16, 24, kernel_size=(5, 5), stride=1, padding=1),
                        nn.BatchNorm2d(24),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2),
                        nn.Conv2d(24, 32, kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=(3, 3), stride=3),
        )
        
        self.layer_3 = nn.Linear(16 * 16 * 32, 910)
#         self.layer_4 = nn.Linear(2730, 910)
        self.layer_5 = nn.Linear(910, 303)
        self.layer_6 = nn.Linear(303, 12)
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = out.reshape(out.shape[0], -1)
        out = self.layer_3(out)
#         out = self.layer_4(F.relu(out))
        out = self.layer_5(F.relu(out))
        out = self.layer_6(F.relu(out))

        return out

In [28]:
num_epochs = 10
net = PlantNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)
total_step = len(train_dloader)

In [29]:
net.train()
for epoch in range(num_epochs):
    for i, (target, sample) in enumerate(train_dloader):
        pred = net(sample)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/250], Loss: 1.7785
Epoch [1/10], Step [200/250], Loss: 1.2821
Epoch [2/10], Step [100/250], Loss: 1.1773
Epoch [2/10], Step [200/250], Loss: 1.1284
Epoch [3/10], Step [100/250], Loss: 0.5772
Epoch [3/10], Step [200/250], Loss: 1.0173
Epoch [4/10], Step [100/250], Loss: 0.4671
Epoch [4/10], Step [200/250], Loss: 0.5629
Epoch [5/10], Step [100/250], Loss: 0.5002
Epoch [5/10], Step [200/250], Loss: 0.4988
Epoch [6/10], Step [100/250], Loss: 0.3532
Epoch [6/10], Step [200/250], Loss: 0.2194
Epoch [7/10], Step [100/250], Loss: 0.0781
Epoch [7/10], Step [200/250], Loss: 0.1430
Epoch [8/10], Step [100/250], Loss: 0.0858
Epoch [8/10], Step [200/250], Loss: 0.1683
Epoch [9/10], Step [100/250], Loss: 0.0779
Epoch [9/10], Step [200/250], Loss: 0.3099
Epoch [10/10], Step [100/250], Loss: 0.0440
Epoch [10/10], Step [200/250], Loss: 0.0052


In [30]:
net.eval()
total = 0
for (target, sample) in valid_dloader:
    pred = net(sample)
    total = total + (torch.argmax(pred, 1) == target).sum().item()

print("Accuracy: {0}%".format(100 * total / len(valid_indices)))

Accuracy: 72.0%
